In [1]:
import tensorflow as tf

In [4]:
train = pd.read_csv("./dataset/train.csv")
test = pd.read_csv("./dataset/test.csv")

In [5]:
y_train = train["label"]
train = train.drop("label", axis=1)
X_train = train

In [6]:
X_train.shape

(42000, 784)

In [8]:
X_train = X_train.values.reshape(-1, 28, 28, 1)

In [18]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

In [20]:
y_train = pd.get_dummies(y_train)

In [22]:
y_train = y_train.values

In [61]:
y_train.shape

(42000, 10)

In [67]:
yy = pd.read_csv("./dataset/train.csv")["label"]

In [68]:
yy_dummies = pd.get_dummies(yy)

In [70]:
label = yy_dummies.columns

In [52]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # 32개의 커널을 가진 컨볼루션 계층을 만듦
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME') # 커널 슬라이딩 시 이미지의 가장 외곽에서 한 칸 밖으로 움직임
L1 = tf.nn.relu(L1)
# 풀링 계층
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], 
                    padding='SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], 
                    padding='SAME')

W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [87]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# epoch = 50
batch_size = 100

train_size = X_train.shape[0]
total_batch = int(train_size / batch_size)

# 시간이 오래 걸려 colab에서 실행
for epoch in range(10):
    total_cost = 0
    
    idx = np.random.permutation(train_size)
    X_random = X_train[idx]
    y_random = y_train[idx]
    
    for i in range(total_batch):
        batch_xs = X_random[i * total_batch:(i+1) * total_batch]
        batch_ys = y_random[i * total_batch:(i+1) * total_batch]
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, 
                                                             Y: batch_ys, 
                                                             keep_prob: 0.7})
        total_cost += cost_val        

    print("Epoch: {:04d}".format(epoch + 1), 
          "Avg. cost: {:.3f}".format(total_cost / total_batch))
    
print()
print("Optimization completed!")

Epoch: 0001 Avg. cost: nan
Epoch: 0002 Avg. cost: nan
Epoch: 0003 Avg. cost: nan
Epoch: 0004 Avg. cost: nan
Epoch: 0005 Avg. cost: nan
Epoch: 0006 Avg. cost: nan
Epoch: 0007 Avg. cost: nan
Epoch: 0008 Avg. cost: nan
Epoch: 0009 Avg. cost: nan
Epoch: 0010 Avg. cost: nan

Optimization completed!


In [95]:
X_test = test.values.reshape(-1, 28, 28, 1)
pred = tf.argmax(model, 1)
prediction = sess.run(pred, feed_dict={X: X_test, keep_prob: 1})
prediction = pd.Series(prediction, name="Label")
submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"), prediction],axis = 1)
submission.to_csv("tensorflow_cnn.csv", index=False)

In [ ]:
epoch = 50
batch_size = 100
train_size = X_train.shape[0]
total_batch = int(train_size / batch_size)

for e in range(epoch):
    idx = numpy.random.permutation(train_size)
    batch_xs = X_train[idx]
    batch_ys = y_train[idx]
    for i in range(total_batch):
        batch_X = X_random[i * BATCH_SIZE:(i+1) * BATCH_SIZE]
        batch_Y = Y_random[i * BATCH_SIZE:(i+1) * BATCH_SIZE]
        sess.run(train,feed_dict = {X: batch_X, Y:batch_Y})

In [ ]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

L1 = tf.layers.conv2d(X, 32, [3, 3])
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(L1, 64, [3, 3])
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=None)
L3 = tf.layers.dropout(L3, 0.7, is_training)

model = tf.layers.dense(L3, 10, activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], 
                               feed_dict={X: batch_xs, 
                                          Y: batch_ys, 
                                          is_training: True})
        total_cost += cost_val
    
    print("Epoch: {:04d}".format(epoch + 1), 
          "Avg. cost: {:.3f}".format(total_cost / total_batch))
    
print()
print("Optimization completed!")

In [ ]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("Accuracy: ", sess.run(accuracy, 
                             feed_dict={X: mnist.test.images.reshape(
                                 -1, 28, 28, 1), 
                                        Y: mnist.test.labels, 
                                        is_training: False}))